## Resultat för kurser (YH-utbildningar upp till 99 poäng)

### 2024

In [5]:
from pathlib import Path

import pandas as pd

DATA_PATH = "data"

current_dir = globals()["_dh"][0]  # CWD for jupyter
path_data = Path(current_dir) / DATA_PATH

In [6]:
# EXTRACT

df_2024 = pd.read_excel(path_data / "resultat-2024-for-kurser-inom-yh.xlsx", sheet_name=0, skiprows=0)
df_2024["Ansökningsomgång"] = 2024


# TRANSFORM

# Standardization of 'Beslut' and convert to bool
df_2024["Beslut"] = df_2024["Beslut"].map({"Beviljad": True, "Ej beviljad": False, "Avslag": False})
if df_2024["Beslut"].isna().sum() > 0:
    raise ValueError(f"Found {df_2024['Beslut'].isna().sum()} NaN values in 'Beslut'")


# Create bool column "Flera kommuner" from column "Kommun"
df_2024["Flera kommuner"] = False
df_2024.loc[df_2024["Kommun"] == 'Se "Lista flera kommuner"', "Flera kommuner"] = True


# Replace "Se \"Lista flera kommuner\"" with <NA>
df_2024["Kommun"] = df_2024["Kommun"].replace('Se "Lista flera kommuner"', pd.NA)
df_2024["Län"] = df_2024["Län"].replace('Se "Lista flera kommuner"', pd.NA)


# Sum and remove column "Antal beviljade platser start och slut 2024"
df_2024["Antal beviljade platser start 2024"] = (
    df_2024["Antal beviljade platser start 2024"] + df_2024["Antal beviljade platser start och slut 2024"]
)
df_2024 = df_2024.drop(columns=["Antal beviljade platser start och slut 2024"])


# Expand approved columns with year in column "Start"
years = [2024, 2025]
new_rows = []
for _, row in df_2024.iterrows():
    rows_added = False

    for year in years:
        column_name = f"Antal beviljade platser start {year}"
        if row[column_name] > 0:
            new_row = row.copy()
            new_row["Beviljade platser"] = row[column_name]
            new_row["Start"] = year
            new_rows.append(new_row)
            rows_added = True

    if not rows_added:
        new_row = row.copy()
        new_row["Beviljade platser"] = 0
        new_row["Start"] = years[0]
        new_rows.append(new_row)

df_2024 = pd.DataFrame(new_rows)


# Drop, rename, reorder
df_2024 = df_2024.drop(
    columns=[
        "Antal beviljade platser start 2024",
        "Antal beviljade platser start 2025",
        "FA-region",
        "Antal län",
        "Antal FA-regioner",
    ]
)

df_2024 = df_2024.rename(
    columns={
        "Anordnare namn": "Utbildningsanordnare",
        "Totalt antal beviljade platser": "Beviljade platser totalt",
    }
)

cols_order = [
    "Diarienummer",
    "Ansökningsomgång",
    "Beslut",
    "Utbildningsanordnare",
    "Utbildningsnamn",
    "Utbildningsområde",
    "YH-poäng",
    "Beviljade platser",
    "Beviljade platser totalt",
    "Start",
    "Län",
    "Kommun",
    "Antal kommuner",
    "Flera kommuner",
]
df_2024 = df_2024[cols_order]


# LOAD

df_2024.to_csv(path_data / "resultat-for-kurser-inom-yh-2024-beslut.csv", index=False)


# RESULT

df_2024.sample(10)

,Diarienummer,Ansökningsomgång,Beslut,Utbildningsanordnare,Utbildningsnamn,Utbildningsområde,YH-poäng,Beviljade platser,Beviljade platser totalt,Start,Län,Kommun,Antal kommuner,Flera kommuner
872,MYH 2024/990,2024,True,Företagsuniversitetet AB,Upphandling av digitaliserings- och IT-lösningar,"Ekonomi, administration och försäljning",15,50,50,2024,Stockholm,Stockholm,1,False
525,MYH 2024/1557,2024,True,Stiftelsen PROTEKO - Nordiska Textilakademin,Design för en cirkulär textilbransch,"Kultur, media och design",90,20,40,2024,Västra Götaland,Borås,1,False
375,MYH 2024/1405,2024,True,"Stockholms stad, Frans Schartaus Handelsinstitut",Juridik inom hälso-och sjukvård för vårdadmini...,"Ekonomi, administration och försäljning",20,30,60,2024,Stockholm,Stockholm,1,False
775,MYH 2024/876,2024,True,Stiftelsen Stockholms Tekniska Institut,Järnvägsprojektering,Samhällsbyggnad och byggteknik,50,35,70,2024,Stockholm,Stockholm,1,False
738,MYH 2024/819,2024,True,Folkuniversitetet - Stiftelsen Kursverksamhete...,IT-Säkerhet Microsoft Azure,Data/IT,40,22,22,2024,<NA>,<NA>,3,True
241,MYH 2024/1261,2024,False,Lexicon Yrkeshögskola AB,Frontendutveckling med Vue.js,Data/IT,35,0,0,2024,Stockholm,Stockholm,1,False
765,MYH 2024/858,2024,False,EC Utbildning AB Stockholm,Rust,Data/IT,40,0,0,2024,Stockholm,Stockholm,1,False
451,MYH 2024/1482,2024,True,One Academy AB,Dialog och förhandling i offentlig upphandling,"Ekonomi, administration och försäljning",15,40,80,2024,<NA>,<NA>,2,True
153,MYH 2024/1169,2024,False,Plushögskolan AB - Affärshögskolan,"KMA (Kvalite, Miljö- och Arbetsmiljö)",Samhällsbyggnad och byggteknik,30,0,0,2024,<NA>,<NA>,3,True
798,MYH 2024/903,2024,True,IHM Business School AB Göteborg,AI-driven hållbarhetsutveckling,"Ekonomi, administration och försäljning",50,40,80,2024,Västra Götaland,Göteborg,1,False


In [7]:
# EXTRACT

df_2024a = pd.read_excel(
    path_data / "resultat-2024-for-kurser-inom-yh.xlsx",
    sheet_name=0,
    skiprows=0,
    usecols=["Diarienummer", "Kommun", "Beslut"],
)

df_2024b = pd.read_excel(
    path_data / "resultat-2024-for-kurser-inom-yh.xlsx",
    sheet_name=2,
    skiprows=0,
    usecols=["Diarienummer", "Kommun"],
)


# TRANSFORM

# Map "Beslut" column to "Flera kommuner" rows
beslut_mapping = df_2024a.set_index("Diarienummer")["Beslut"].to_dict()
df_2024b["Beslut"] = df_2024b["Diarienummer"].map(beslut_mapping)


# Drop reffering rows
df_2024a = df_2024a[df_2024a["Kommun"] != 'Se "Lista flera kommuner"']


df_diarie_kommun = pd.concat([df_2024a, df_2024b], ignore_index=True)
df_diarie_kommun = df_diarie_kommun.drop_duplicates().sort_values(by="Diarienummer").reset_index(drop=True)


# Standardization of 'Beslut' and convert to bool
df_diarie_kommun["Beslut"] = df_diarie_kommun["Beslut"].map({"Beviljad": True, "Ej beviljad": False, "Avslag": False})
if df_diarie_kommun["Beslut"].isna().sum() > 0:
    raise ValueError(f"Found {df_diarie_kommun['Beslut'].isna().sum()} NaN values in 'Beslut'")


# LOAD

df_diarie_kommun.to_csv(path_data / "resultat-for-kurser-inom-yh-2024-diarie_kommun.csv", index=False)


# RESULT

df_diarie_kommun


,Diarienummer,Beslut,Kommun
0,MYH 2024/1000,False,Stockholm
1,MYH 2024/1000,False,Boden
2,MYH 2024/1000,False,Malmö
3,MYH 2024/1001,False,Malmö
4,MYH 2024/1001,False,Boden
...,...,...,...
1197,MYH 2024/997,False,Nyköping
1198,MYH 2024/998,True,Nyköping
1199,MYH 2024/999,False,Malmö
1200,MYH 2024/999,False,Boden


In [8]:
df_antal = pd.read_excel(
    path_data / "resultat-2024-for-kurser-inom-yh.xlsx",
    sheet_name=0,
    skiprows=0,
    usecols=["Antal kommuner"],
)

sum_antal_kommuner = df_antal["Antal kommuner"].sum()

if sum_antal_kommuner != len(df_diarie_kommun):
    raise AssertionError(
        f"Sum of 'Antal kommuner' ({len(df_diarie_kommun)}) does not match count of rows in df_diarie_kommun ({count_diarie_kommun})"
    )